In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math
import requests
from io import BytesIO
from datetime import datetime

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl
from plotly.subplots import make_subplots

from PIL import Image

from resources.tier_chart import tier_chart
from resources.plotly_theme import nfl_template
from resources.get_nfl_data import get_team_info, get_pbp_data
from resources.team_stats import get_team_stats
from resources.player_stats import get_player_stats

pio.templates['nfl_template'] = nfl_template

# Process Data

In [2]:
''' Params / Constants '''

## Params ##
THRU_WEEK = 10
EXPORT = False

## Constants ##

LEAGUE_LOGO = 'https://raw.githubusercontent.com/nflverse/nflverse-pbp/master/NFL.png'
PLOT_BUFFER = 0.05

In [3]:
''' Import Data '''

# Import
team_data = get_team_info()
pbp_data = get_pbp_data(years=[2025])

# Filter to current week
league_data = pbp_data.loc[pbp_data['week'] <= THRU_WEEK, :]
week_data = pbp_data.loc[pbp_data['week'] == THRU_WEEK, :].copy()

print(team_data.shape)
print(team_data.head().to_string())
print(league_data.shape)
print(league_data.tail().to_string())
print(week_data.shape)
print(week_data.head().to_string())

2025 done.
Downcasting floats.
(32, 15)
              team_name  team_id  team_nick team_conf team_division team_color team_color2 team_color3 team_color4                                                                                                   team_logo_wikipedia                                          team_logo_espn                                                          team_wordmark                                         team_conference_logo                                                        team_league_logo                                                          team_logo_squared
team                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
gpby = league_data.loc[(league_data['down'] > 1) & (league_data['Play Counted']) & (league_data['Offensive Snap']) & (~league_data['Is Special Teams Play']),:].groupby('posteam').aggregate(
    Plays=('play', 'sum'),
    First=('play', lambda x: x[league_data['down'] == 1].sum()),
    Second=('play', lambda x: x[league_data['down'] == 2].sum()),
    Third=('play', lambda x: x[league_data['down'] == 3].sum()),
    Fourth=('play', lambda x: x[league_data['down'] == 4].sum()),
    OnSchedulePlays=('On Schedule Play', 'sum'),
    OnSchedulePlaysFirst=('On Schedule Play', lambda x: x[league_data['down'] == 1].sum()),
    epa=('epa', 'sum')
)
gpby['On Schedule %'] = gpby['OnSchedulePlays'] / gpby['Plays']
gpby['On Schedule First'] = gpby['OnSchedulePlaysFirst'] / gpby['First']
gpby['epa / play'] = gpby['epa'] / gpby['Plays']
gpby = gpby.sort_values(by='On Schedule %', ascending=False)

print(gpby.to_string())

In [4]:
''' Get Stats '''

## Offense ##

# Overall
team_offense = get_team_stats(league_data, unit='offense')
team_offense = team_offense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(team_offense.sort_values(by='EPA / Play', ascending=False).head().to_string())

# Week
week_offense = get_team_stats(week_data, unit='offense')
week_offense = week_offense.merge(team_data[['team_color', 'team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(week_offense.sort_values(by='EPA / Play', ascending=False).head().to_string())


## Defense ##

# Overall
team_defense = get_team_stats(league_data, unit='defense')
team_defense = team_defense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(team_defense.sort_values(by='EPA / Play', ascending=True).head().to_string())

# Week
week_defense = get_team_stats(week_data, unit='defense')
week_defense = week_defense.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(week_defense.sort_values(by='EPA / Play', ascending=True).head().to_string())

         Games  Plays  OnSchedulePlays   TDs  FirstDowns  ThirdDownAtts  ThirdDownConvs  RushAttempts  RushYards  RushTDs  Rush1Ds  DesignedRushPlays  DesignedRushAttempts  DesignedRushYards  QBScrambles  ScrambleYards  DesignedPassPlays  Dropbacks  PassCompletions  PassAttempts  PassYards  PassTDs  Pass1Ds  Sacks  SackYards  INTs  TFLs  Fumbles  Penalties  PenaltyYards  Penalty1Ds  Drives  Total Yards  Turnovers  PlaysAdv  PassPlays  RushPlays         EPA    RushEPA    PassEPA  Successes  RushSuccesses  PassSuccesses       WPA   RushWPA   PassWPA  EPA / Play  Rush EPA / Play  Pass EPA / Play  Success Rate  Rush Success Rate  Pass Success Rate                                     team_logo_espn                                                          team_wordmark
posteam                                                                                                                                                                                                                           

# League Averages

In [12]:
print(f'Success Rate: {team_offense["Successes"].sum() / team_offense["PlaysAdv"].sum():.2%}')
print(f'Passing Success Rate: {team_offense["PassSuccesses"].sum() / team_offense["PassPlays"].sum():.2%}')
print(f'Rushing Success Rate: {team_offense["RushSuccesses"].sum() / team_offense["RushPlays"].sum():.2%}')
print()
print(f'EPA / Play: {team_offense["EPA"].sum() / team_offense["PlaysAdv"].sum():.2f}')
print(f'Pass EPA / Play: {team_offense["PassEPA"].sum() / team_offense["PassPlays"].sum():.2f}')
print(f'Rush EPA / Play: {team_offense["RushEPA"].sum() / team_offense["RushPlays"].sum():.2f}')


Success Rate: 44.97%
Passing Success Rate: 47.14%
Rushing Success Rate: 41.44%

EPA / Play: 0.03
Pass EPA / Play: 0.08
Rush EPA / Play: -0.05


## Rushing

In [ ]:
''' Rushing Tiers '''


fig = tier_chart(
    data_frame=team_offense,
    y_col='Rush Success Rate',
    x_col='Rush EPA / Play',
    logos_col='team_logo_espn',
    n_tiers=7,
    title=f'<b>NFL Rushing Tiers</b><br><sup>Thru Week {THRU_WEEK}</sup>'
)

fig.update_layout_images(sizex=0.03, sizey=0.03)
fig.update_yaxes(
    tickformat='.0%',
)
fig.update_xaxes(
    tickformat='.2f'
)
fig.update_layout(
    height=500, width=700
)
fig.show()

if EXPORT: pio.write_image(fig, f'Rushing Tiers - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

In [ ]:
''' Rushing Profiles '''


fig = go.Figure()

## Data
X = team_offense['RushAttempts'].to_numpy()
Y = team_offense['Rush EPA / Play'].to_numpy()
LOGOS = team_offense['team_logo_espn'].to_numpy()

x_range = X.max() - X.min()
y_range = Y.max() - Y.min()

## Scatter
fig = px.scatter(
    x=X,
    y=Y,
)

# Logos
for i in range(len(X)):
    fig.add_layout_image(
        source=LOGOS[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=X[i], # X-coordinate of the image's center
        y=Y[i], # Y-coordinate of the image's center
        sizex=6,   # Width of the image in data units
        sizey=6,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.9
    )

# Quadrant annotations
fig.add_annotation(
    text='RUSH A LOT<br>GOOD RUSHING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.max() - (y_range*PLOT_BUFFER), 
    x=X.max() - (x_range*PLOT_BUFFER),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    text='RUSH A LOT<br>BAD RUSHING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.min() + (y_range*PLOT_BUFFER), 
    x=X.max() - (x_range*PLOT_BUFFER),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    text='DON\'T RUSH A LOT<br>GOOD RUSHING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.max() - (y_range*PLOT_BUFFER), 
    x=X.min() + (x_range*PLOT_BUFFER),
    align='center',
    # font=dict(color="#467bd0", weight='bold')
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    text='DON\'T RUSH A LOT<br>BAD RUSHING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.min() + (y_range*PLOT_BUFFER), 
    x=X.min() + (x_range*PLOT_BUFFER),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
    
## Format
VIZ_X_RANGE = (X.min() - (x_range*PLOT_BUFFER), X.max() + (x_range*PLOT_BUFFER))
VIZ_Y_RANGE = (Y.min() - (y_range*PLOT_BUFFER), Y.max() + (y_range*PLOT_BUFFER))

fig.add_vline(x=X.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')
fig.add_hline(y=Y.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')

fig.update_traces(
    marker=dict(opacity=0)
)
fig.update_yaxes(
    title='EPA / Play',
    range=VIZ_Y_RANGE,
    tickformat='.2f'
)
fig.update_xaxes(
    title='Attempts',
    range=VIZ_X_RANGE,
    tickformat=','
)
fig.update_layout(
    template="nfl_template",
    title=dict(
        text=f'<b>NFL Team Rushing Profiles</b><br><sup>Thru Week {THRU_WEEK}</sup>'
    ),
    showlegend=False,
    height=500, width=700
)

# Credits
fig.add_annotation(
    text=f'Figure: @jmillerr240 | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.1, 
    x=1,
    align='left'
)
fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Rushing Profiles - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

## Passing

In [ ]:
''' Passing Tiers '''

fig = tier_chart(
    data_frame=team_offense,
    y_col='Pass Success Rate',
    x_col='Pass EPA / Play',
    logos_col='team_logo_espn',
    title=f'<b>NFL Passing Tiers</b><br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
)

fig.update_yaxes(
    tickformat='.0%'
)
fig.update_xaxes(
    tickformat='.2f'
)
fig.update_layout_images(sizex=0.04, sizey=0.04)
fig.update_layout(width=700, height=500)
fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Passing Tiers - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

In [ ]:
''' Passing Profile '''

fig = go.Figure()


## Data
X = team_offense['PassAttempts'].to_numpy()
Y = team_offense['Pass EPA / Play'].to_numpy()
LOGOS = team_offense['team_logo_espn'].to_numpy()

x_range = X.max() - X.min()
y_range = Y.max() - Y.min()

## Scatter
fig = px.scatter(
    x=X,
    y=Y,
)

# Logos
for i in range(len(X)):
    fig.add_layout_image(
        source=LOGOS[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=X[i], # X-coordinate of the image's center
        y=Y[i], # Y-coordinate of the image's center
        sizex=8,   # Width of the image in data units
        sizey=8,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.9
    )

# Quadrant annotations
fig.add_annotation(
    # text='Pass a lot and are good at it',
    text='PASS A LOT<br>GOOD PASSING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.max() - (y_range*PLOT_BUFFER), 
    x=X.max() - (x_range*PLOT_BUFFER),
    # y=Y.mean() + (y_range*0.4), 
    # x=X.mean() + (x_range*0.25),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    # text='Pass a lot but are bad at it',
    text='PASS A LOT<br>BAD PASSING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    # y=Y.mean() - (y_range*0.4), 
    # x=X.mean() + (x_range*0.25),
    y=Y.min() + (y_range*PLOT_BUFFER), 
    x=X.max() - (x_range*PLOT_BUFFER),
    align='center',
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    # text='Don\'t pass a lot but are good at it',
    text='DON\'T PASS A LOT<br>GOOD PASSING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.max() - (y_range*PLOT_BUFFER), 
    x=X.min() + (x_range*PLOT_BUFFER),
    # y=Y.mean() + (y_range*0.4), 
    # x=X.mean() - (x_range*0.25),
    align='center',
    # font=dict(color="#467bd0", weight='bold')
    font=dict(color="#646c76", weight='bold')
)
fig.add_annotation(
    # text='Don\'t pass a lot and are bad at it',
    text='DON\'T PASS A LOT<br>BAD PASSING TEAM',
    showarrow=False,
    xref='x',
    yref='y',
    y=Y.min() + (y_range*PLOT_BUFFER), 
    x=X.min() + (x_range*PLOT_BUFFER),
    # y=Y.mean() - (y_range*0.4), 
    # x=X.mean() - (x_range*0.25),
    align='center',
    font=dict(color="#646c76", weight='bold')
)

## Format
VIZ_X_RANGE = (X.min() - (x_range*PLOT_BUFFER), X.max() + (x_range*PLOT_BUFFER))
VIZ_Y_RANGE = (Y.min() - (y_range*PLOT_BUFFER), Y.max() + (y_range*PLOT_BUFFER))

fig.add_vline(x=X.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')
fig.add_hline(y=Y.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')

fig.update_traces(
    marker=dict(opacity=0)
)
fig.update_yaxes(
    title='EPA / Play',
    range=VIZ_Y_RANGE,
    tickformat='.2f'
)
fig.update_xaxes(
    title='Attempts',
    range=VIZ_X_RANGE,
    tickformat=','
)
fig.update_layout(
    template="nfl_template",
    title=dict(
        text=f'<b>NFL Team Passing Profiles</b><br><sup>Thru Week {THRU_WEEK}</sup>'
    ),
    showlegend=False,
    width=700, height=500
)

# Credits
fig.add_annotation(
    text=f'Figure: @jmillerr240 | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.1, 
    x=1,
    align='left'
)
fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Passing Profiles - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

In [ ]:
''' Offense Tiers '''

## Chart
fig = tier_chart(
    data_frame=team_offense,
    x_col='Pass EPA / Play',
    y_col='Rush EPA / Play',
    logos_col='team_logo_espn',
    title=f'<b>NFL Offense Tiers</b><br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
)

fig.update_xaxes(
    tickformat='0.2f'
)
fig.update_yaxes(
    tickformat='0.2f'
)
fig.update_layout_images(sizey=0.06)
fig.update_layout(width=700, height=500)
fig.update_annotations(y=-.1)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Offense Tiers - Thru Week {THRU_WEEK}.png', scale=6, width=700, height=500)

In [ ]:
''' Offense Ranks '''

## Data ##
offense_ranks = team_offense.copy().rename(columns={
    'RushYards': 'Rush Yards',
    'PassYards': 'Pass Yards',
    'FirstDowns': 'First Downs',
    'Fumbles': 'Fumbles Lost',
})

# Rank
data_cols = ['Plays', 'Pass Yards', 'Rush Yards', 'First Downs', 'TDs', 'INTs', 'Fumbles Lost', 'Sacks', 'EPA / Play']
rank_cols = [f'{col}_rank' for col in data_cols]

for col in data_cols:
    if col in ['INTs', 'Fumbles', 'Sacks']:
        offense_ranks[f'{col}_rank'] = offense_ranks[col].rank(axis=0, method='max', ascending=True)
    else:
        offense_ranks[f'{col}_rank'] = offense_ranks[col].rank(axis=0, method='min', ascending=False)

offense_ranks = offense_ranks[data_cols + rank_cols]

# Logos
offense_ranks = offense_ranks.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(offense_ranks.sort_values(by='EPA / Play', ascending=False).head().to_string())


## Viz ##

# Data
x = data_cols
y = offense_ranks.index.to_numpy()
z = offense_ranks[rank_cols].to_numpy()
text = offense_ranks[rank_cols].astype(int).astype(str).to_numpy()
wordmarks = offense_ranks['team_wordmark'].tolist()

# Heat map
heat_map = go.Heatmap(
    x=x, y=y, z=z, 
    text=text, texttemplate="%{text}", textfont=dict(size=10, color='white'),
    colorscale='PRGn', #['navy', '#c8c8c8', '#4bb543'], #'Cividis',
    reversescale=True,
    showscale=False,
    colorbar=dict(
        lenmode='fraction',
        len=0.5,
        title=dict(
            text='Rank',
            font=dict(weight='bold')
        ),
        xref='paper', yref='paper',
        x=1.1, xanchor='right'
    ),
    xgap=2, ygap=2
)

# Create figure
fig = go.Figure()

fig.add_trace(heat_map)

# Logos
for i in range(len(wordmarks)):
# for i in range(2):
    response = requests.get(wordmarks[i])
    logo_img = Image.open(BytesIO(response.content))
    fig.add_layout_image(
        x=-0.075,
        y=(1-(1/64))-((1/32)*i),
        sizex=.125,
        sizey=.125,
        xanchor='center',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=logo_img,
    )
    
# Format
fig.update_xaxes(
    side='top',
    showgrid=False,
    ticks='',
    tickfont=dict(weight='bold'),
)
fig.update_yaxes(
    autorange='reversed',
    showgrid=False,
    ticks='',
    visible=False,
)
fig.update_layout(
    template='nfl_template',
    title=dict(
        text=f'<b>NFL Offensive Ranks</b><br><sup>Thru Week {THRU_WEEK}</sup>',
        yref='container', y=0.965, yanchor='top',
        x=0.035
    ),
    margin=dict(t=75, b=25, l=100, r=25),
    height=900,
    width=700,
    margin_pad=5
)

# Credits
fig.add_annotation(
    text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    xref='paper', yref='paper',
    x=1, y=-.025,
    showarrow=False,
    align='right'
)

fig.show()

# Export
if EXPORT: pio.write_image(fig, file=f'NFL Offensive Ranks - Thru Week {THRU_WEEK}.png', height=900, width=700, scale=6)

In [ ]:
''' EPA by Field Position '''

## Calc Data
# Filter out special teams
pbp_field_pos = league_data.loc[~league_data['Is Special Teams Play'],:].copy()

# Round yardlines
pbp_field_pos['yardline_rnd'] = np.ceil(pbp_field_pos['yardline_100'] / 5)*5

# Group by yardline
by_yardline = pbp_field_pos.groupby(['posteam', 'yardline_rnd']).aggregate(
    Plays=('posteam', 'size'),
    EPA=('epa', 'sum')
)
totals = by_yardline.groupby(['yardline_rnd']).aggregate(
    Plays=('Plays', 'sum'),
    EPA=('EPA', 'sum')
)
totals.index = pd.MultiIndex.from_product([['League'], totals.index])
by_yardline = pd.concat([by_yardline, totals])

print(by_yardline.loc[by_yardline.index.get_level_values(0)=='League'].to_string())


## Heat maps

MAX_EPA = by_yardline.loc[by_yardline.index.get_level_values(0) != 'League', 'EPA'].max()
MIN_EPA = by_yardline.loc[by_yardline.index.get_level_values(0) != 'League', 'EPA'].min()
print(MIN_EPA, MAX_EPA)

# Field
img = Image.open('./resources/field white.jpg')
img_width, img_height = img.size
ratio = img_width / img_height

# Yardlines
yardlines_raw = by_yardline.index.get_level_values(1).unique()
yardlines = np.where(yardlines_raw > 50, '-'+(50 - (yardlines_raw - 50)).astype(int).astype(str), '+'+yardlines_raw.astype(int).astype(str)).tolist()

# Divisions
divisions = team_data.reset_index().set_index(['team_division', 'team_abbr']).sort_index()
divisions = divisions.loc[divisions.index.get_level_values(1).isin(pbp_data['posteam'].unique()), :]
print(divisions.head().to_string())


## Create heatmaps
for division in divisions.index.get_level_values(0).unique():
    division_sl = divisions.loc[divisions.index.get_level_values(0) == division, :]
    teams = division_sl.index.get_level_values(1).sort_values().tolist()
    conf_logo = division_sl['team_conference_logo'].unique().tolist()[0]

    logos = []
    nick_names = []
    total_epas = []
    heat_maps = []
    for team in teams:
        # Team
        df = by_yardline.loc[by_yardline.index.get_level_values(0) == team, :].copy()
        
        # Logo
        team_info = team_data.loc[team_data.index == team, :]
        name = team_info['team_nick'].values[0]
        logo = team_info['team_logo_espn'].values[0]

        # Fill in missing yardlines
        for y in yardlines_raw:
            if y not in df.index.get_level_values(1):
                df.loc[(team, y), :] = [0, 0]
        df = df.sort_index(ascending=True)

        # Data inputs
        x = [team for i in range(len(yardlines))] #by_yardline.index.get_level_values(0)
        y = yardlines                       #y=colts_epa.index.get_level_values(1)
        z = df['EPA'].tolist()

        # Heatmap
        heat_map = go.Heatmap(
            x=y,
            y=x,
            z=z,
            opacity=0.7,
            # opacity=1,
            coloraxis='coloraxis',
            zmid=0
        )

        heat_maps.append(heat_map)
        total_epas.append(df['EPA'].sum())
        logos.append(logo)
        nick_names.append(name)

    N_COLS = 2
    N_ROWS = 2
    titles = []
    for i in range(len(teams)):
        title = f'{nick_names[i]} ({total_epas[i]:,.1f} Total EPA)'
        titles.append(title)

    ## Build Figure
    fig = make_subplots(rows=N_ROWS, cols=N_COLS, subplot_titles=titles)
    fig.update_annotations(font=dict(size=14))

    SIZEX = 0.45 #0.2885
    X_SPACING = (1 - (SIZEX*N_COLS)) / (N_COLS - 1)

    SIZEY = 0.375
    Y_SPACING = (1 - (SIZEY*N_ROWS)) / (N_ROWS - 1)

    i = 0
    for r in range(1, N_ROWS+1):
        for c in range(1, N_COLS+1):
            # Heat map
            fig.add_trace(
                heat_maps[i],
                row=r, col=c
            )

            # Field
            x = (SIZEX*(c-1)) + (X_SPACING*(c-1))
            y = 1 - ((SIZEY*(r-1)) + (Y_SPACING*(r-1)))
            fig.add_layout_image(
                x=x,
                y=y,
                sizey=SIZEY,
                sizex=SIZEX,
                xref="paper",
                yref="paper",
                opacity=1,
                layer="below",
                # opacity=0.2,
                # layer="above",
                sizing="stretch",
                source=img
            )

            # Logo in middle of field
            xref = 'x' if i == 0 else f'x{i+1}'
            yref = 'y' if i == 0 else f'y{i+1}'
            fig.add_layout_image(
                x=9.5,
                y=0,
                sizey=4,
                sizex=4,
                xref=xref,
                yref=yref,
                opacity=0.6,
                layer="above",
                sizing="contain",
                yanchor='middle',
                xanchor='center',
                source=logos[i]
            )

            i += 1

    # Conference Logo
    response = requests.get(conf_logo)
    conf_logo_img = Image.open(BytesIO(response.content))
    fig.add_layout_image(
        x=1,
        y=1.2,
        sizex=.15,
        sizey=.15,
        xanchor='right',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=conf_logo_img,
    )

    # Direction annotation
    fig.add_annotation(
        text='<i>Moving left to right</i>',
        font=dict(size=12),
        xref='paper', yref='paper',
        x=0.5, y=0.5,
        showarrow=False
    )

    ## Formatting
    fig.update_xaxes(
        type='category',
        categoryorder='array', 
        categoryarray=yardlines,
        autorange='reversed',
        showgrid=False,
        visible=False,
    )
    fig.update_yaxes(
        showticklabels=False,
        showgrid=False,
        visible=False
    )

    r = max(abs(MIN_EPA), abs(MAX_EPA))
    fig.update_coloraxes(
        cmin=-1*r,
        cmax=r,
        cmid=0,
        colorscale=["#ae0d0a", 'white', "#297523"],
        colorbar_title=dict(text='Total EPA', font=dict(weight='bold'))
    )
    fig.update_layout(
        template='nfl_template',
        title=dict(
            text=f'Total <span style="color: #3f9938">EPA</span> by Field Position<br><sup>{division}</sup>'
        ),
        autosize=False,
        height=500,
        width=900,
    )

    # Credits
    fig.add_annotation(
        text=f'Thru Week {THRU_WEEK}<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
        xref='paper', yref='paper',
        x=1, y=-.15,
        showarrow=False,
        align='right'
    )

    fig.show()

    # pio.write_image(fig, file=f'EPA by Field Position - {division} - Week {THRU_WEEK}.png', scale=6, height=500, width=900)


# Defense

In [ ]:
''' Rush D Tiers '''

fig = tier_chart(
    data_frame=team_defense,
    y_col='Rush Success Rate',
    x_col='Rush EPA / Play',
    logos_col='team_logo_espn',
    title=f'<b>NFL Rushing Defense Tiers</b><br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
    x_reversed=True,
    y_reversed=True
)

fig.update_yaxes(
    tickformat='.0%'
)
fig.update_xaxes(
    tickformat='.2f'
)
fig.update_layout_images(sizex=0.03, sizey=0.03)
fig.update_annotations(y=-.1)
fig.update_layout(width=700, height=500)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Rushing Defense Tiers - Thru Week {THRU_WEEK}.png',scale=6, width=700, height=500)

In [ ]:
''' Pass D Tiers '''

fig = tier_chart(
    data_frame=team_defense,
    y_col='Pass Success Rate',
    x_col='Pass EPA / Play',
    logos_col='team_logo_espn',
    title=f'<b>NFL Passing Defense Tiers</b><br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
    x_reversed=True,
    y_reversed=True
)


fig.update_yaxes(
    tickformat='.0%'
)
fig.update_xaxes(
    tickformat='.2f'
)
fig.update_layout_images(sizex=0.035, sizey=0.035)
fig.update_annotations(y=-.1)
fig.update_layout(width=700, height=500)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Passing Defense Tiers - Thru Week {THRU_WEEK}.png',scale=6, width=700, height=500)

In [ ]:
''' Defense Tiers '''

## Chart
fig = tier_chart(
    data_frame=team_defense,
    x_col='Pass EPA / Play',
    y_col='Rush EPA / Play',
    logos_col='team_logo_espn',
    title=f'<b>NFL Defense Tiers</b><br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
    x_reversed=True,
    y_reversed=True
)

fig.update_xaxes(
    tickformat='0.2f'
)
fig.update_yaxes(
    tickformat='0.2f'
)
fig.update_layout_images(sizey=0.05)
fig.update_annotations(y=-.1)
fig.update_layout(width=700, height=500)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Defense Tiers - Thru Week {THRU_WEEK}.png',scale=6, width=700, height=500)

In [ ]:
''' EPA / Snaps '''


## Create Scatter
X = team_defense['Plays'].to_numpy()
Y = team_defense['EPA / Play'].to_numpy()

scatter = px.scatter(
    x=X,
    y=Y,
    trendline='ols',
    trendline_color_override='#44546a',
    opacity=0.8
)
# Access the R-squared value
r_squared_value = px.get_trendline_results(scatter).px_fit_results.iloc[0].rsquared
print(f"R-squared value: {r_squared_value}")


## Create Figure
fig = go.Figure()

# Scatter
for trace in scatter.data:
    fig.add_trace(trace)

# Avg Lines
fig.add_vline(x=X.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')
fig.add_hline(y=Y.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='above')

# Logos
LOGOS = team_defense['team_logo_espn'].tolist()
for i in range(len(team_defense)):
    fig.add_layout_image(
        source=LOGOS[i],  # The loaded image
        xref="x",    # Reference x-coordinates to the x-axis
        yref="y",    # Reference y-coordinates to the y-axis
        x=X[i], # X-coordinate of the image's center
        y=Y[i], # Y-coordinate of the image's center
        sizex=9,   # Width of the image in data units
        sizey=9,   # Height of the image in data units
        xanchor="center", # Anchor the image by its center horizontally
        yanchor="middle", # Anchor the image by its middle vertically
        layer="above", # Place image above other plot elements
        opacity=0.9
    )

# Format
fig.update_xaxes(
    title='Snaps',
    tickformat=','
)
fig.update_yaxes(
    title='EPA / Play',
    tickformat='.2f',
    autorange='reversed'
)
fig.update_layout(
    template='nfl_template',
    title=f'Defensive EPA by Snaps<br><sup>Thru Week {THRU_WEEK}</sup>',
    width=900, height=500
)
fig.update_traces(
    marker=dict(opacity=0),
)

# Credits
fig.add_annotation(
    text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    xref='paper', yref='paper',
    x=1, y=-.1,
    showarrow=False,
    align='right'
)

fig.show()

In [ ]:
''' Defense Ranks '''

## Data ##
defense_ranks = team_defense.copy().rename(columns={
    'RushYards': 'Rush Yards',
    'PassYards': 'Pass Yards',
    'FirstDowns': '1Ds',
    'Fumbles': 'Fumbles Rec.',
})

# Rank
data_cols = ['Plays', 'Pass Yards', 'Rush Yards', '1Ds', 'TDs', 'INTs', 'Fumbles Rec.', 'Sacks', 'TFLs', 'EPA / Play']
rank_cols = [f'{col}_rank' for col in data_cols]

for col in data_cols:
    if col in ['Pass Yards', 'Rush Yards', '1Ds', 'TDs', 'EPA / Play']:
        defense_ranks[f'{col}_rank'] = defense_ranks[col].rank(axis=0, method='max', ascending=True)
    else:
        defense_ranks[f'{col}_rank'] = defense_ranks[col].rank(axis=0, method='min', ascending=False)

defense_ranks = defense_ranks[data_cols + rank_cols]

# Logos
defense_ranks = defense_ranks.merge(team_data[['team_logo_espn', 'team_wordmark']], left_index=True, right_index=True)

print(defense_ranks.sort_values(by='EPA / Play', ascending=True).head().to_string())


## Viz ##

# Data
x = data_cols
y = defense_ranks.index.to_numpy()
z = defense_ranks[rank_cols].to_numpy()
text = defense_ranks[rank_cols].astype(int).astype(str).to_numpy()
wordmarks = defense_ranks['team_wordmark'].tolist()

# Heat map
heat_map = go.Heatmap(
    x=x, y=y, z=z, 
    text=text, texttemplate="%{text}", textfont=dict(size=10, color='white'),
    colorscale='PRGn', #['navy', '#c8c8c8', '#4bb543'], #'Cividis',
    reversescale=True,
    showscale=False,
    colorbar=dict(
        lenmode='fraction',
        len=0.5,
        title=dict(
            text='Rank',
            font=dict(weight='bold')
        ),
        xref='paper', yref='paper',
        x=1.1, xanchor='right'
    ),
    xgap=2, ygap=2
)

# Create figure
fig = go.Figure()

fig.add_trace(heat_map)

# Logos
for i in range(len(wordmarks)):
# for i in range(2):
    response = requests.get(wordmarks[i])
    logo_img = Image.open(BytesIO(response.content))
    fig.add_layout_image(
        x=-0.075,
        y=(1-(1/64))-((1/32)*i),
        sizex=.125,
        sizey=.125,
        xanchor='center',
        yanchor='middle',
        xref='paper', 
        yref='paper',
        source=logo_img,
    )
    
# Format
fig.update_xaxes(
    side='top',
    showgrid=False,
    ticks='',
    tickfont=dict(weight='bold'),
)
fig.update_yaxes(
    autorange='reversed',
    showgrid=False,
    ticks='',
    visible=False,
)
fig.update_layout(
    template='nfl_template',
    title=dict(
        text=f'<b>NFL Defensive Ranks</b><br><sup>Thru Week {THRU_WEEK}</sup>',
        yref='container', y=0.965, yanchor='top',
        x=0.035
    ),
    margin=dict(t=75, b=25, l=100, r=25),
    height=900,
    width=700,
    margin_pad=5
)

# Credits
fig.add_annotation(
    text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    xref='paper', yref='paper',
    x=1, y=-.025,
    showarrow=False,
    align='right'
)

fig.show()

# Export
if EXPORT: pio.write_image(fig, file=f'NFL Defensive Ranks - Thru Week {THRU_WEEK}.png', height=900, width=700, scale=6)

# Team

In [ ]:
''' Team Tiers '''

## Data
team_df = team_offense[['EPA / Play']].merge(team_defense[['EPA / Play', 'team_logo_espn']], left_index=True, right_index=True)
team_df = team_df.rename(columns={
    'EPA / Play_x': 'Offensive EPA / Play',
    'EPA / Play_y': 'Defensive EPA / Play',
})

## Chart
fig = tier_chart(
    data_frame=team_df,
    x_col='Offensive EPA / Play',
    y_col='Defensive EPA / Play',
    logos_col='team_logo_espn',
    title=f'<b>NFL Team Tiers</b><br><sup>Thru Week {THRU_WEEK}</sup>',
    n_tiers=7,
    x_reversed=False,
    y_reversed=True
)

fig.update_xaxes(
    tickformat='0.2f'
)
fig.update_yaxes(
    tickformat='0.2f'
)
fig.update_layout_images(sizey=0.04)
fig.update_annotations(y=-.1)
fig.update_layout(width=700, height=500)

fig.show()

# Export
if EXPORT: pio.write_image(fig, f'Team Tiers - Thru Week {THRU_WEEK}.png',scale=6, width=700, height=500)

In [ ]:
''' Third Downs '''

# Filter to 3rd down attempts
third_down_sl = league_data.loc[((league_data['third_down_converted'] == 1) | (league_data['third_down_failed'] == 1)), :]

third_down_off = third_down_sl.groupby('posteam').aggregate(
    ThirdDownAtt=('posteam', 'size'),
    ThirdDownOnScheduleAtt=('On Schedule Play', 'sum'),
    ThirdConv=('third_down_converted', 'sum'),
    ThirdFail=('third_down_failed', 'sum'),
    ThirdConvPass=('third_down_converted', lambda x: x[(third_down_sl['penalty'] == 0) & (third_down_sl['pass_attempt'] == 1)].sum()),
    ThirdConvRun=('third_down_converted', lambda x: x[(third_down_sl['penalty'] == 0) & (third_down_sl['rush_attempt'] == 1)].sum()),
    ThirdConvPen=('third_down_converted', lambda x: x[third_down_sl['penalty'] == 1].sum()),
    ThirdDownYards=('yards_gained', 'sum'),
    ThirdDownPassYards=('passing_yards', 'sum'),
    ThirdDownRushYards=('rushing_yards', 'sum'),
    ThirdDownTDs=('touchdown', 'sum'),
    ThirdDownINTs=('interception', 'sum'),
    ThirdDownFumblesLost=('fumble_lost', 'sum'),
    ThirdDownSacks=('sack', 'sum'),
    ThirdDownSackYards=('yards_gained', lambda x: x[third_down_sl['sack'] == 1].sum()),
    ThirdDownEPA=('epa', 'sum'),
    ThirdDownWPA=('wpa', 'sum'),
)
third_down_off['ThirdDownOnScheduleRate'] = third_down_off['ThirdDownOnScheduleAtt'] / third_down_off['ThirdDownAtt']
third_down_off['Conversion %'] = third_down_off['ThirdConv'] / third_down_off['ThirdDownAtt']
third_down_off['Conversion % Pass'] = third_down_off['ThirdConvPass'] / third_down_off['ThirdDownAtt']
third_down_off['Conversion % Run'] = third_down_off['ThirdConvRun'] / third_down_off['ThirdDownAtt']
third_down_off['Conversion % Pen'] = third_down_off['ThirdConvPen'] / third_down_off['ThirdDownAtt']
third_down_off = third_down_off.sort_values(by='Conversion %', ascending=False)

print(third_down_off['ThirdDownAtt'].sum())
print(third_down_off['ThirdConv'].sum())
print(third_down_off['ThirdConv'].sum() / third_down_off['ThirdDownAtt'].sum())
print('TDs:', third_down_off['ThirdDownTDs'].sum())
print('Sack yards:', third_down_off['ThirdDownSackYards'].sum())
print('EPA:', third_down_off['ThirdDownEPA'].sum())

third_down_def = third_down_sl.groupby('defteam').aggregate(
    ThirdDownAtt=('posteam', 'size'),
    ThirdDownOnScheduleAtt=('On Schedule Play', 'sum'),
    ThirdConv=('third_down_converted', 'sum'),
    ThirdFail=('third_down_failed', 'sum'),
    ThirdConvPass=('third_down_converted', lambda x: x[(third_down_sl['penalty'] == 0) & (third_down_sl['pass_attempt'] == 1)].sum()),
    ThirdConvRun=('third_down_converted', lambda x: x[(third_down_sl['penalty'] == 0) & (third_down_sl['rush_attempt'] == 1)].sum()),
    ThirdConvPen=('third_down_converted', lambda x: x[third_down_sl['penalty'] == 1].sum()),
)
third_down_def['ThirdDownOnScheduleRate'] = third_down_def['ThirdDownOnScheduleAtt'] / third_down_def['ThirdDownAtt']
third_down_def['Conversion %'] = third_down_def['ThirdConv'] / third_down_def['ThirdDownAtt']
third_down_def['Conversion % Pass'] = third_down_def['ThirdConvPass'] / third_down_def['ThirdDownAtt']
third_down_def['Conversion % Run'] = third_down_def['ThirdConvRun'] / third_down_def['ThirdDownAtt']
third_down_def['Conversion % Pen'] = third_down_def['ThirdConvPen'] / third_down_def['ThirdDownAtt']
third_down_def = third_down_def.sort_values(by='Conversion %', ascending=False)

# print(third_down_def['ThirdDownAtt'].sum())
# print(third_down_def['ThirdDownOnScheduleAtt'].sum())

third_down_eff = third_down_off.merge(third_down_def, left_index=True, right_index=True, suffixes=('_off', '_def'))
third_down_eff = third_down_eff.merge(team_data[['team_color', 'team_logo_espn']], left_index=True, right_index=True)

# print(third_down_eff['ThirdConv_off'].sum() / third_down_eff['ThirdDownAtt_off'].sum())

print(third_down_eff.to_string())

In [ ]:
third_down_team = get_team_stats(league_data.loc[(league_data['down'] == 3),:].copy(), unit='offense').copy()
third_down_team['On Sch. Rate'] = third_down_team['OnSchedulePlays'] / third_down_team['Plays']
third_down_team['Conversion Rate'] = third_down_team['ThirdDownConvs'] / third_down_team['ThirdDownAtts']

print(third_down_team['Plays'].sum())
print(third_down_team['ThirdDownAtts'].sum())
print(third_down_team['ThirdDownConvs'].sum())
print(third_down_team['ThirdDownConvs'].sum() / third_down_team['ThirdDownAtts'].sum())
print('TDs:', third_down_team['TDs'].sum())
print('Sack yards:', third_down_team['SackYards'].sum())
print('EPA:', third_down_team['EPA'].sum())

print(third_down_team.sort_values(by='Conversion Rate', ascending=False).to_string())

In [ ]:
''' Third Down Efficiency '''

for unit in ['off', 'def']:
    third_down_eff = third_down_eff.sort_values(by=f'Conversion %_{unit}', ascending=False if unit == 'off' else True)
    
    ## Column ##
    x = third_down_eff.index.tolist()
    y = third_down_eff[f'Conversion %_{unit}'].to_numpy() 
    logos = third_down_eff['team_logo_espn'].tolist()

    # Create figure
    fig = go.Figure(data=[
        go.Bar(name='Pass', x=x, y=third_down_eff[f'Conversion % Pass_{unit}'].tolist(), text=third_down_eff[f'ThirdConvPass_{unit}'].astype(int).tolist(),
               marker=dict(opacity=0.7, color='#013369', line=dict(color='#323232', width=1))
        ),
        go.Bar(name='Run', x=x, y=third_down_eff[f'Conversion % Run_{unit}'].tolist(), text=third_down_eff[f'ThirdConvRun_{unit}'].astype(int).tolist(), 
               marker=dict(opacity=0.7, color='#2B8C4E', line=dict(color='#323232', width=1))
        ),
        go.Bar(name='Penalty', x=x, y=third_down_eff[f'Conversion % Pen_{unit}'].tolist(), text=third_down_eff[f'ThirdConvPen_{unit}'].astype(int).tolist(), 
               marker=dict(opacity=0.7, color='#D50A0A', line=dict(color='#323232', width=1))
        ),
    ])

    # Avg line
    fig.add_hline(y=y.mean(), line_width=1, line_dash="dash", line_color="#CB4747", layer='below')

    # # Logos
    for i in range(len(logos)):
        fig.add_layout_image(
            source=logos[i],  # The loaded image
            xref="x",    # Reference x-coordinates to the x-axis
            yref="y",    # Reference y-coordinates to the y-axis
            x=x[i], # X-coordinate of the image's center
            y=y[i], # Y-coordinate of the image's center
            sizex=0.9,   # Width of the image in data units
            sizey=0.9,   # Height of the image in data units
            xanchor="center", # Anchor the image by its center horizontally
            yanchor="middle", # Anchor the image by its middle vertically
            layer="above", # Place image above other plot elements
            opacity=0.8
        )

    # Format
    fig.update_xaxes(
        ticks='',
        visible=False
    )
    fig.update_yaxes(
        tickformat='.0%',
        title='Conversion %'
    )
    fig.update_traces(
        insidetextanchor='middle',
    )
    fig.update_layout(
        template='nfl_template',
        title=f'<b>Third Down {"Offense" if unit == "off" else "Defense"}</b><br><sup>Thru Week {THRU_WEEK}</sup>',
        barmode='stack',
        legend=dict(x=0.9 if unit == 'off' else 0.05, bgcolor='rgba(0,0,0,0)'),
        margin=dict(t=50, b=25, r=25, l=40),
        width=900, height=500
    )

    # Credits
    fig.add_annotation(
        text=f'Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
        showarrow=False,
        xref='paper',
        yref='paper',
        y=-0.05, 
        x=1,
        align='right'
    )

    fig.show()

    # Export
    if EXPORT: pio.write_image(fig, f'Third Down {"Offense" if unit == "off" else "Defense"} - Thru Week {THRU_WEEK}.png',
                               scale=6, width=900, height=500)

# Players

In [ ]:
''' Player Stats '''

player_stats = get_player_stats(pbp_data=league_data)
player_stats = player_stats.sort_values(by='Total EPA', ascending=False)

print(player_stats.head().to_string())

In [ ]:
''' Player Contributions - Offense '''


for position in ['QB', 'WR', 'RB', 'TE']:

    min_plays = 144 if position == 'QB' else 30

    viz_sl = player_stats.loc[(player_stats['position'] == position) & (player_stats['Total Plays'] >= min_plays), :]

    ## Data ##
    x = viz_sl['Total EPA'].to_numpy()
    # y = viz_sl.index.get_level_values(1)
    y = viz_sl['player_id'].tolist()
    text = viz_sl.index.get_level_values(1).tolist()
    logos = viz_sl['team_logo_espn'].tolist()
    colors = viz_sl['team_color'].tolist()
    headshots = viz_sl['headshot'].tolist()

    n_players = len(x)

    x_range = x.max() - x.min()

    ## Bar ##
    bar = px.bar(
        x=x,
        y=y,
        text=text
    )

    fig = go.Figure()

    for trace in bar.data:
        fig.add_trace(trace)

    # Logos
    for i in range(len(logos)):
        x_pos = x[i] - (x_range*.03) if x[i] < 0 else x[i] + (x_range*.03)
        fig.add_layout_image(
            # x=-0.05,
            # y=(1-((1/n_players)*.5))-((1/(n_players))*i),
            x=x_pos, y=y[i],
            # sizex=(1/n_players)*.9,
            # sizey=(1/n_players)*.9,
            sizex=x_range*.05,
            sizey=x_range*.05,
            xanchor='center',
            yanchor='middle',
            xref='x', 
            yref='y',
            source=logos[i],
            opacity=0.8,
            layer='below'
        )

    # Format
    fig.update_traces(marker=dict(opacity=0.7, color=colors, line=dict(width=1.5, color='black')))
    fig.update_yaxes(
        autorange='reversed',
        visible=False
    )
    fig.update_xaxes(
        title='Total EPA'
    )
    fig.update_layout(
        template='nfl_template',
        title=dict(
            text=f'<b>Offensive Contributions: {position}s</b><br><sup>Total EPA on offensive plays, passing + receiving + rushing; min. {min_plays} plays; Thru week {THRU_WEEK}</sup>',
            yref='container', y=0.97, yanchor='top'
        ),
        height=1000, width=700,
        margin=dict(t=60, r=25, b=50, l=25)
    )

    # Credits
    fig.add_annotation(
        text=f'<br>Figure: @clankeranalytic | Data: nfl_data_py | {datetime.today().strftime("%Y-%m-%d")}',
        showarrow=False,
        xref='paper',
        yref='paper',
        y=-0.05, 
        x=1,
        align='right'
    )

    fig.show()

    # Export
    if EXPORT: pio.write_image(fig, f'Offensive Contributions: {position}s - Week {THRU_WEEK}.png',
                               scale=6, height=1000, width=700)

In [ ]:
print(player_stats.loc[player_stats.index.get_level_values(1) == 'M.Harrison',:].to_string())

In [ ]:
marv = league_data.loc[(league_data['receiver'] == 'M.Harrison'), ['down', 'ydstogo', 'desc', 'pass_attempt', 'penalty', 'first_down', 'first_down_penalty', 'success', 'Successful Play', 'epa']]

print(f'Plays: {marv.shape[0]}')
print(f'Tgts: {marv["pass_attempt"].sum()}')
print(f'Success: {marv["success"].sum()}')
print(f'Success Plays: {marv["Successful Play"].sum()}')
print(f'EPA: {marv["epa"].sum()}')
print(f'Pen: {marv["penalty"].sum()}')
print(f'1D Pen: {marv["first_down_penalty"].sum()}')

print(marv.to_string())